In [1]:
import json
import gzip
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np
import torch
import tensorflow as tf

from transformers import T5ForConditionalGeneration, PreTrainedTokenizerFast

In [2]:
config_file = 'config.json'
pytorch_model_path = 'converted_pt_model'
model = T5ForConditionalGeneration.from_pretrained(pytorch_model_path)

In [3]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
print(f'{pytorch_total_params:,}')

1,141,824,512


In [4]:
model.generation_config

GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0
}

In [5]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('VBARTTokenizer_T5_Sentinels')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

# Test tokenizer's sentinel tokens
for i in range(32001, 32004):
    print(tokenizer.decode([i]))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Device: cpu
<extra_id_98>
<extra_id_97>
<extra_id_96>


/home/meliksah.turker/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


- Infer on Onur's Test Dataset **Tokens**

In [6]:
# Read Data
json_test_data_path = "/media/disk/datasets/bounllm/oscarmc4_cleaned_hf_dataset_validation-single_shards_with_encoded_values.json.gz"
gzip_file = gzip.open(json_test_data_path)
byte_lines = [line for line in gzip_file]
json_lines = [json.loads(byte_line) for byte_line in byte_lines]

In [7]:
sample_idx = 33
inputs = json_lines[sample_idx]['inputs']
targets = json_lines[sample_idx]['targets']

print(inputs)
print('--------------')
print(targets)
inputs_encoded = json_lines[sample_idx]['inputs_encoded']
targets_encoded = json_lines[sample_idx]['targets_encoded']

model_inputs = tf.keras.utils.pad_sequences([inputs_encoded], maxlen = 512, padding = 'post', truncating = 'post')
model_inputs = torch.from_numpy(model_inputs).to(device)

[NLU] gelmez. Bu açıdan sürekliliği olan bu tip programları devam <extra_id_0> önemlidir. Burada emeği geçenlere teşekkür ediyorum. Yığinki, neticede hepimizin çocukluğunun geçtiği, lahanasından is <extra_id_1>na <extra_id_2> yetiştirdiği insanlara kadar <extra_id_3> kadim bir mahalleden bahsediyoruz. Elazığ'ı Aksaray'sız düşünmek mümkün değil. Bu mahalle <extra_id_4> komşuluk kültüründen öte bir akraba kültürü ile <extra_id_5> insanlardır. Yığ <extra_id_6>ki’yi yeniden anmak <extra_id_7> gelmek, elbette büyük bir mutluluktur” <extra_id_8>. Yapısal, fonksiyonel ve sanatsal özellikleriyle iç ve dış mekanda kullanılan bir mimari şeklidir. Demirin kullanılmasıyla elde edilen ferforjeler, göze hoş gelen ve yapımı ince işçilik isteyen ürünlerdir. Bu konuda hizmet veren firmalar, özellikle nostaljik görüntüden <extra_id_9> iç mimaride tercih edilmektedir. “Ferforje” konusunda iç mimaride ayna çerçevesi, yatak başlığı, şamdanlık, merdiven <extra_id_10>, dış mekanlarda ise bina kapısı, bahçe k

In [8]:
model_outputs = model.generate(inputs = model_inputs, max_length = 512, do_sample = False)
model_output_as_list = model_outputs.numpy()[0].tolist()
print(tokenizer.decode(model_output_as_list))

<PAD><extra_id_0> ettirmek<extra_id_1>kambil oynadığı, torunları<extra_id_2>,<extra_id_3> herkesin<extra_id_4>,<extra_id_5> yaşayan<extra_id_6>in<extra_id_7>, “Bu mahalleye<extra_id_8> demek istiyorum<extra_id_9> uzak<extra_id_10>i<extra_id_11> kapı ve pencereler gibi birçok aksesuar bulunmaktadır. Burada dikkat<extra_id_12> bu<extra_id_13> eski<extra_id_14>,<extra_id_15> sahip olmak<extra_id_16> gereklidir. Bu<extra_id_17> sonra<extra_id_18>forje” konusunda üretim<extra_id_19> son yıllarda bu konuda<extra_id_20> daha<extra_id_21>”<extra_id_22> Bu sayede hem iç hem de dış cephedeki demirin renk<extra_id_23> daha kaliteli<extra_id_24>” de<extra_id_25>lar<extra_id_26> “Ferforje” konusunda yapılan çalışmalar ve çözüm<extra_id_27>de yer aldığını belirten Karaman,<EOS>


- Same input but instead of using the tokens provided in the json file, tokenize the text input via **VBARTTokenizer_T5_Sentinels** tokenizer and then infer.

In [9]:
inputs_encoded_vbart_tokenizer = tokenizer.encode(inputs)
targets_encoded_vbart_tokenizer = tokenizer.encode(targets)

model_inputs = tf.keras.utils.pad_sequences([inputs_encoded_vbart_tokenizer], maxlen = 512, padding = 'post', truncating = 'post')
model_inputs = torch.from_numpy(model_inputs).to(device)

In [10]:
model_outputs = model.generate(inputs = model_inputs, max_length = 512, do_sample = False)
model_output_as_list = model_outputs.numpy()[0].tolist()
print(tokenizer.decode(model_output_as_list))

<PAD><extra_id_0> ettirebilmek<extra_id_1>kambil<extra_id_2>ğme<extra_id_3> çok<extra_id_4>,<extra_id_5> yaşayan<extra_id_6>in<extra_id_7>, onun için<extra_id_8> dedi<extra_id_9> uzak<extra_id_10>, balkon<extra_id_11> balkon gibi kullanım alanları bulunmaktadır. Burada dikkat<extra_id_12> bu<extra_id_13> eski<extra_id_14>,<extra_id_15> sahip olmak<extra_id_16> gereklidir. Bu<extra_id_17> sonra<extra_id_18>forje” konusunda üretim<extra_id_19> son yıllarda üretim konusunda<extra_id_20> daha<extra_id_21>”<extra_id_22> Bu sayede hem iç hem dış cephe<extra_id_23> daha işlevsel<extra_id_24>”<extra_id_25>lar<extra_id_26> “Ferforje” konusunda çözüm<extra_id_27>, “Ferforje” konusunda<extra_id_28>”<extra_id_29> sahip olmak da önemlidir. Bu<extra_id_30> sahip olmak<extra_id_31><EOS>


- Infer with custom text input. Using samples from **pretrain_oscarmc4_cleaned_hf_dataset-predict.txt**

In [11]:
sample_1 = "[NLG] G <extra_id_0> katılımıyla yedi sayısına <extra_id_1>. Bir yıl sonra AB de toplantılara katılmaya başlamış, G <extra_id_2> adı yine yedi rakamıyla birlikte anılmakta <extra_id_3> zirvede Trump konuyu gündeme getirmişti <extra_id_4> Bu yıl da gündeme getirdiği, İngiltere, Almanya, Kanada ve <extra_id_5>nın itirazlarına rağmen 2020 <extra_id_6> katılımcılar arasında <extra_id_7>işim açısından <extra_id_8> Sorular çözülmese <extra_id_9> planlı ve plansız toplantıların da sorunların yönetimine ve çözümüne katkıda bulunduğunu unutmamakta <extra_id_10> var. 24-26 Ağustos’ta Fransa’nın Biarritz kentinde düzenlenen bu yılki zirvede de pek <extra_id_11> masaya yatır <extra_id_12> savaşları, küresel ısınma, uluslararası güvenlik, İran ve Amazon ormanlarındaki yangınlar farklı oturumlarda <extra_id_13> alındı. <extra_id_14> sorunu henüz çözülmedi ancak Trump’ın açıklamalarının tonu değişti. Cumhurbaşkanı Ruhani ile <extra_id_15> olgunlaşırsa <extra_id_16>’ın şartlardan ne kast ettiği, dönüş yolunda ya da <extra_id_17> fikrini değiştirip değiştirmeyeceği tabii ki bilinmiyor <extra_id_18> Her an her şey olabilir, Trump <extra_id_19> yaptırım paketini uygulamaya koyup Zarif ve Ruhani’nin manevra alanını daraltmayı da deneyebilir. İran <extra_id_20> nihayetinde bizim üstümüze de yük oluyor. <extra_id_21> başka önemli konu da Amerika’nın Rusya’yı yeniden G7 platformuna dahil <extra_id_22>, diğer komşumuz <extra_id_23> konuşabilecekleri zeminlerin sayısının artması Türkiye’nin genel olarak lehine. Diğer yandan bu yakınlaşmanın Suriye’de kurduğumuz zor denklemleri etkileme potansiyeli taşıdığını da görmezden <extra_id_24> Amerika-Rusya yakınlaşması dünyadaki gerginliği <extra_id_25> olduğunda Türkiye’nin çıkar ve beklentilerini dikkate almayan bir <extra_id_26> büyük devlet uzlaşmaları çok yaşandı. <extra_id_27> G7 Zirvesi’nden çıkartması gereken bir başka sonuç ise <extra_id_28>’nin <extra_id_29> kabul ettiği Trablus Hükümeti üstündeki baskı <extra_id_30> Zirve Deklarasyonunda da zaten Libya için tüm tarafları <extra_id_31> bir uluslararası konferans çağrısı yapıldı ve Afrika Birliği <extra_id_32>lendiği vurgulandı. <extra_id_33>k <extra_id_34> var ama onların yorumu iktisatçıların alanına giriyor. <extra_id_35> kaydedilir ve <extra_id_36> engeller kalkar. Trump’ın açıklaması, yorumlar ve analizler olumlu yönde. Çin’le <extra_id_37>ebileceğini varsaymakta yarar var diye <extra_id_38> ö <extra_id_39>. O zaman ona göre bir planlama yapmam gerekir... Buna ekstra olarak bir şirket üzerinden <extra_id_40>niz gerekir vergi <extra_id_41>le sizin uğraşmanız eğer acemi <extra_id_42>. Bu <extra_id_43> aynı motorunu 2 <extra_id_44> alım <extra_id_45> iş değişir... Diyelim almanyada tanıdığın var. B <extra_id_46> misal. Türkiye vatandaşı ise, <extra_id_47>la türkiyeye gelir. belli <extra_id_48>. 6 ayda bir yurt dışına çıkması gerekir motorun. Ama gene ucuza mal etmiş <extra_id_49>san <extra_id_50>sin... Bu bilgiler benim 3 <extra_id_51>dir. Yasalarda ve yürütme de değişiklik oldu ise affola... bu <extra_id_52> değiştimi acaba. kıbrıstan araba getiril <extra_id_53>liyordu ama <extra_id_54> biliyorum <extra_id_55>cam"
sample_2 = "[NLU] G7, yani Yediler Grubu dünyanın en gelişmiş yedi ekonomisini bünyesinde barındıran <extra_id_0> ilk kez 1973’de düşünülmüş, dört üye diye başlanmış ama çok geçmeden sayı altıya çıkmış, 1976’da <extra_id_1>nın katılımıyla yedi sayısına ulaşılmış. Bir yıl sonra AB de toplantılara katılmaya başlamış, G8 haline 1997 yılından sonra Rusya Federasyonu’ <extra_id_2> üstüne üye olarak katılımıyla dönüşmüş. Kırım’ın işgali yüzünden Rusya’nın dışlanmasıyla 2014’den bu yana grubun adı yine yedi rakamı <extra_id_3> anılmakta <extra_id_4> Kanada’da düzenlenen olaylı zirvede Trump konuyu gündeme getirmişti. Bu yıl <extra_id_5> getirdiği, İngiltere, Almanya, Kanada ve Fransa’nın itirazlarına rağmen 2020 zirvesinde Putin’i katılımcılar <extra_id_6> olacağı söyleniyor. Ki bu da küresel yönetişim açısından önemli. Sorular çözülmese <extra_id_7>lerin ve <extra_id_8> planlı ve <extra_id_9>sız toplantıların da sorunların yönetimine ve çözümüne katkıda bulunduğunu <extra_id_10>. 24-26 Ağustos’ta Fransa’nın Biarritz kentinde düzenlenen bu yılki zirvede de pek çok küresel sorun masaya yatırıldı. Resmi <extra_id_11> olan toplantıda ticaret savaşları, küresel ısınma, uluslararası güvenlik <extra_id_12> ve Amazon ormanlarındaki yangınlar farklı oturumlarda ele alındı. İran da belli ki sesini duyurabildi. <extra_id_13>di ancak Trump’ın açıklamalarının tonu <extra_id_14>. Cumhurbaşkanı Ruhani ile şartlar olgunlaşırsa buluşabileceğini söyledi. Trump’ın şartlardan ne kast ettiği, dönüş yolunda ya da gelecek <extra_id_15>yeceği tabii ki bilinmiyor. Her an her şey <extra_id_16>, Trump ve yönetimi her şeyi yapabilir. Yeni bir yaptırım paketini uygulamaya koyup Zarif ve Ruhani <extra_id_17> daraltmayı da deneyebilir. İran’a uygulanan yaptırımlar nihayetinde bizim üstümüze de yük oluyor. Bir başka önemli konu da Amerika’nın Rusya’yı yeniden G7 platformuna <extra_id_18> Biri müttefik <extra_id_19> İki büyük ülkenin yakınlaşması, sorunlarını konuşabilecekleri zeminlerin sayısının artması Türkiye’nin genel olarak lehine. Diğer yandan bu yakınlaşmanın Suriye’de kurduğumuz zor denklemleri etkileme potansiyeli taşıdığını da görmezden gelemeyiz. Amerika-Rusya yakınlaşması dünyadaki gerginliğin ve çatışma risklerinin azaltılmasına yardımcı <extra_id_20> Suriye söz konusu olduğunda Türkiye’ <extra_id_21> çıkar ve beklentilerini dikkate <extra_id_22> bir uzlaşmaya var <extra_id_23> yol açabilir. Ne de olsa <extra_id_24> “quid pro quo” olarak adlandırılan bu tür büyük devlet uzlaşmaları çok yaşandı. Türkiye’ <extra_id_25> çıkartması gereken bir başka sonuç ise Libya’da çözüm için taraflar <extra_id_26> daha doğrusu Türkiye’nin desteklediği, BM’in meşru kabul ettiği Trablus Hükümeti üstündeki baskının artma olasılığının yükseldiği. Zirve Deklarasyonunda da zaten Libya için tüm tarafları ve bölgesel aktörleri kapsayan bir uluslararası konferans çağrısı yapıldı ve Afrika Birliği ile BM’nin çabalarının desteklendiği"
sample_3 = "[NLU] az 3 harf girerek arama yapabilirsiniz. Kitaplarında bir samimiyet ve sahicilik var. Kimi zaman anlatıcı ile sohbet ediyormuşum <extra_id_0>. Yazarken karşında okur olarak tahayyül ettiğin hayali ya da gerçek birileri var mı? Böyle <extra_id_1>ne sevindim, gerçi ben yazarken doğruyu söylediğimi hissetmiyorum. Yalan söylediğimi de hissetmiyorum. Belki bir çeşit kaygan gerçekliğe ilerliyorum ama bu hiçbir zaman bitmeyen bir sürecin parçası. Yazarken okurları pek düşünmüyorum. Her zaman yazdıklarımın kontrolünü kaybedeceğim bir noktaya varmak istiyorum. Metnin kendi <extra_id_2>nı <extra_id_3> Bazı yazarlar <extra_id_4> da <extra_id_5> yazmaya eğilimliyken, bazıları da uzun soluklu romanlarla kendilerini daha rahat ifade edebiliyor. Sence bunun beynimizin bilişsel işlevleri ya da ayarlarıyla bir ilgisi var mı? Yazacağın metnin biçimini önceden planlar mısın <extra_id_6> mı bırakır <extra_id_7> böyle düşünmemiştim. Şu anda uzun bir romanı yarıladım, o zaman belki beynime zıt yönde çalışıyorum! Herkes bir romanın nasıl olması gerektiği hakkında konuşuyor ama ben kitabın kendi kurallarını koyduğuna <extra_id_8> yontmak gibi. Büyük bir kayan var ve içinde gizlenmiş bir kitabın, yazmak ise onu bulmak <extra_id_9> Tek veya birden fazla <extra_id_10> ile başlıyorum. Çok fazla taslak yazıyorum. Kitap yazdığımı unutmaya çalışıyorum. Son kitabın Multiple Choice’u 1993 <extra_id_11> Şili’de girdiğin çoktan seçmeli üniversite sınavının biçiminde kurgulamışsın. Okur metni anlamlandırma sürecine <extra_id_12> dâhil oluyor, aynı zamanda yaratıcı düşünceyi körelten eğitim sistemiyle dalga geçiyorsun. Testin yaratıcı bir edebi <extra_id_13>ya dönüşmesi epey ironik. Deneysel edebiyatın yayıncılık sektöründeki kabulünde, okuldaki geleneksel hocalarımızın şüpheciliği ve direncine benzer bir durum var mı sence? Ama bu normal, bütün <extra_id_14>. Şunu belirtmeliyim ki, deneysel edebiyat olarak algılanılacak bir şey yazmak için uğraşmıyordum. Sadece anlatmaya çalıştığım şeyin o biçimde anlatılması gerektiğine ikna oldum. Edebiyat <extra_id_15> zorlar <extra_id_16> daha tuhaftır ama Multiple Choice önceki kitaplarımdan daha deneyselmiş gibi hissetmiyorum. Öyle gözüküyor ama benim <extra_id_17> değil. Kitabı, kapağındaki test sorusundan da anlaşıldığı gibi, <extra_id_18>ye koyamıyoruz, hepsinden birer parça barındırıyor. Edebi türler <extra_id_19>? <extra_id_20> her zaman bulanık <extra_id_21>. Edebiyat tarihi bundan ibaret. Kitap pazarı sınırları <extra_id_22> kitaplara ürün gibi davranıyor ama eminim ki okurlar bunu düşünmüyor. Edebi türler etikettir veya kılavuz niteliği taşır ama her iyi kitap buna karşı koyar. Büyülü Dağ’ı okuduktan <extra_id_23> ‘ <extra_id_24>, bu bir roman’ <extra_id_25>dim. Sadece bütünlüklü ve anlamlı bir deneyim olduğunu <extra_id_26> Bence edebi türler, giydiğin ama bir türlü rahat edemediğin gömlekler gibi"

In [12]:
for sample in [sample_1, sample_2, sample_3]:
    inputs_encoded_vbart_tokenizer = tokenizer.encode(sample)
    
    model_inputs = tf.keras.utils.pad_sequences([inputs_encoded_vbart_tokenizer], maxlen = 512, padding = 'post', truncating = 'post')
    model_inputs = torch.from_numpy(model_inputs).to(device)

    model_outputs = model.generate(inputs = model_inputs, max_length = 512, do_sample = False)
    model_output_as_list = model_outputs.numpy()[0].tolist()
    print(tokenizer.decode(model_output_as_list))
    print('---------------------------------------------------')

<PAD><extra_id_0>7 Zirvesi’nde Türkiye’nin<extra_id_1> kadar görüşmelere devam etti<extra_id_2>7 Zirvesi’nde Türkiye<extra_id_3>ydı. Bu<extra_id_4>.<extra_id_5> Kanada’<extra_id_6> yılında Türkiye’nin G7 Zirvesi’nde<extra_id_7> yer alması<extra_id_8> çok önemli.<extra_id_9> de Türkiye’nin<extra_id_10> yarar<extra_id_11> çok konu<extra_id_12>ıldı. Suriye<extra_id_13> ele<extra_id_14> Türkiye’nin Suriye<extra_id_15> görüşme<extra_id_16> Trump<extra_id_17> Türkiye’nin<extra_id_18>.<extra_id_19> da<extra_id_20>’ın Türkiye’ye karşı yaptırımları<extra_id_21> Bir<extra_id_22> etmesi<extra_id_23> İran’ın da Türkiye’ye karşı<extra_id_24> gelemeyiz.<extra_id_25>n artmasına neden oldu. Türkiye’nin Suriye’de bir güç merkezi<extra_id_26> tutum içinde olması da çok doğal. Türkiye’nin<extra_id_27> Türkiye’nin<extra_id_28>, Türkiye<extra_id_29> de<extra_id_30>nın sona ermesi oldu.<extra_id_31> ikna edecek<extra_id_32>’nin de bu konferansa katılmayı kabul<extra_id_33> Bu konferansa Türkiye’nin de katıl